# k-Nearest Neighbors (kNN)
This Jupyter notebook summarizes the <a href=#pros>Pros</a> and <a href=#cons>Cons</a> of the k-Nearest Neighbors algorithm and gives two Python examples on usage for <a href=#clas>Classification</a> and <a href=#reg>Regression</a>. 

## Theory<sup>1,2,3</sup>  
* Is a non-probabilistic, non-parametric and instance-based learning algorithm (see <a href=#reference>References</a>:
    * **Non-parametric** means it makes no explicit assumptions about the function form of _h_, avoiding the dangers of mis-modelling the underlying distribution of the data
        * For example, suppose our data is highly non-Gaussian but the learning model was choose assumes a Gaussian form. In that case, a parametric algorithm would make extremely poor predictions.
    * **Instance-based** learning means that the algorithm does not explicitly learn a model
        * Instead, it chooses to memorize the training instances which are subsequently used as "knowledge" for the prediction phase
        * Concretely, this means that only when a query to our database is made (i.e., when we ask it to predict a label given an input), will the algorithm use the training instances to predict the result

### Pros<a name="pros"/> 
* **simple** to understand and implement
* with **little to zero training time**
* kNN **works just as easily with multi-class data** sets whereas other algorithms are hard-coded for the binary setting
* the non-parametric nature of kNN gives it an edge in certain settings where the data may be highly unusual, thus **without prior knowledge on distribution**

### Cons<a name="cons"/> 
* **computationally expensive** testing phase
    * we **need to store the whole data set for each decision**!
* can **suffer from skewed class distributions**
    * for example, if a certain class is very frequent in the training set, it will tend to dominate the majority voting of the new example (large number = more common)
* the accuracy can be severally **degraded with high-dimension data** because of the little difference between the nearest and farthest neighbor
    * **the curse of dimensionality** refers to various phenomena that arise when analyzing and organizing data in high-dimensional spaces that do not occur in low-dimensional settings such as the three-dimensional physical space of everyday experience
    * for high-dimensional data (e.g., with number of dimensions more than 10) **scaling** and **dimension reductions** (such as PCA) is usually performed prior applying kNN
    
### References<a name="reference"/>  
* <sup>1</sup>Wikipedia [kNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm), [Curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) 
* <sup>2</sup>Sklearn [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
* <sup>3</sup>[Complete Guide to K-Nearest-Neighbors](https://kevinzakka.github.io/2016/07/13/k-nearest-neighbor)

## Classification<a name="clas"/> 
* the output is a class membership
* an object is classified by a **majority vote** of its neighbours, with the object being assigned to the class most common among its k nearest neighbours
    * if k = 1, then the object is simply assigned to the class of that nearest neighbour
    
    
### Example: predict [IRIS](https://scikit-learn.org/stable/datasets/index.html#iris-dataset) class

Set environment

In [ ]:
# Scikit-learn
from sklearn import datasets
from sklearn.model_selection import train_test_split, GridSearchCV , cross_val_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.metrics import classification_report, mean_squared_error,r2_score, mean_absolute_error,accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.datasets import load_wine
import seaborn as sns
from sklearn.datasets import load_diabetes

In [ ]:
# Use vector drawing inside jupyter notebook
%config InlineBackend.figure_format = "svg"

# Set matplotlib default axis font size (inside this notebook)
plt.rcParams.update({'font.size': 8})

Load data

In [ ]:
iris = datasets.load_iris()

df = pd.DataFrame(iris.data,columns=iris.feature_names)
df = df.assign(target=iris.target)

In [ ]:
df.head()

<div class="alert alert-success">Changed.</div>

In [ ]:
# Check for class distribution in target variable
print("Class distribution:")
print(df['target'].value_counts())

Show data summary: extend the `describe` method by selected stats
* See the Jupyter notebook on **Standard Procedure** for more details

<div class="alert alert-success">Changed.</div>

In [ ]:
# Compute selected stats
dfinfo = pd.DataFrame(df.dtypes,columns=["dtypes"])
for (m,n) in zip([df.count(),df.isna().sum()],["count","isna"]):
    dfinfo = dfinfo.merge(pd.DataFrame(m,columns=[n]),right_index=True,left_index=True,how="inner");


# dfinfo.T.append(df.describe())

dfinfo = pd.concat([dfinfo.T, df.describe()])
dfinfo

Show histogram (distribution)

In [ ]:
plt.figure(figsize=(9,2))
for (i,v) in enumerate(df.columns):
    plt.subplot(1,df.shape[1],i+1);
    plt.hist(df.iloc[:,i],bins="sqrt")
    plt.title(df.columns[i],fontsize=9);

<div class="alert alert-success">Changed.</div>

In [ ]:
# Visualize feature distributions

plt.figure(figsize=(10,6))
df_melted = df.melt(id_vars="target", var_name="Features", value_name="Value")
sns.boxplot(x="Features", y="Value", hue="target", data=df_melted)
plt.xticks(rotation=45)
plt.title("Feature Distributions by Target Class")
plt.show()

Show correlation matrix

In [ ]:
df.corr().round(2).style.background_gradient(cmap="viridis")

Scale and try to **reduce dimensions**: what we try to do is to **always simply the model** if possible (see correlation matrix above)
* More complex model (e.g., more features, or higher _*k*_) will (in theory) increase the probability of higher "out of sample" error (even when "in sample" error = train set) will be smaller!
* Use either 99% threshold (own subjective) or "mle" algorithm (more objective)
* Use **linear** scaler (transformation)
* Here, the data is scaled prior train-test split. 
    * In real applications, first split and scale afterwards, to simulate real-world scenario where we do not have the test set! (otherwise data snooping effect)

<div class="alert alert-success">Changed.</div>

In [ ]:
scaler = StandardScaler()
Xo = scaler.fit_transform(df.drop(columns=["target"]))

In [ ]:
pca = PCA(n_components=0.99)# or set n_components="mle"
X = pca.fit_transform(Xo)
print("Nr. of features after PCA = {} (input = {})".format(X.shape[1],Xo.shape[1]))

Prepare for fitting

In [ ]:
# encode target values (is not necessary for IRIS but still:-)
y = LabelEncoder().fit_transform(df["target"].values);

# Split 2/3 to 1/3 train to test respectively
[X_train,X_test,y_train,y_test] = train_test_split(X,y,train_size = 0.67,test_size = 0.33, stratify=y,random_state=123);

#### Find optimal model
* Considering the small data set (150 samples), find "optimal" k setting it to maximum of 5
    * Optimal in terms of accuracy
    * Simple model = higher probability of lower in and out-of sample error

In [ ]:
model = KNeighborsClassifier(algorithm="auto");
parameters = {"n_neighbors":[1,3,5],
              "weights":["uniform","distance"]}
model_optim = GridSearchCV(model, parameters, cv=5,scoring="accuracy");

In [ ]:
model_optim.fit(X_train,y_train)

Show the "optimal" settings for kNN

In [ ]:
model_optim.best_estimator_

<div class="alert alert-success">Changed.</div>

In [ ]:
for (i,x,y) in zip(["Train","Test"],[X_train,X_test],[y_train,y_test]):
    print("Classification kNN",i," report:\n",classification_report(y,model_optim.predict(x)))

In [ ]:
for i in ["most_frequent","uniform"]:
    dummy = DummyClassifier(strategy=i).fit(X_train,y_train);
    print("Classification ",i," test report:",classification_report(y_test,dummy.predict(X_test)))

In [ ]:
# Try different values of k
k_values = range(1, 21)
error_rates = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=5, scoring='accuracy') 
    error_rates.append(1 - scores.mean())  # Convert accuracy to error rate

# Plot the Elbow Method
plt.figure(figsize=(8,4))
plt.plot(k_values, error_rates, marker='o', linestyle='dashed', color='b')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Error Rate')
plt.title('Elbow Method for Optimal k')
plt.show()

In [ ]:
# Choose best k from the elbow point
optimal_k = k_values[np.argmin(error_rates)]
print(f"Optimal k found: {optimal_k}")


#### Show resulting accuracy

In this case, the precision (accuracy=macro avg precision) is very high. 
Just to show that that is not coincidence compare to "dummy" model (most frequent & uniform distribution)

## Regression<a name="reg"/> 
* Predicts value as the **average of the values** of its k nearest neighbors

### Example: Predict House price
* Use Scikit-learn [California Housing](https://scikit-learn.org/stable/datasets/index.html#california-housing-dataset) data set
    * This is a large data set that allows us to use more complex model
* Nontheless, try to reduce the number of features: via visual inspection and using PCA

Load data

<div class="alert alert-success">Changed.</div>

In [ ]:
house = datasets.fetch_california_housing()
df = pd.DataFrame(house.data,columns=house.feature_names)
df = df.assign(target=house.target)

In [ ]:
df.head()

Inspect data: show statistics, histogram and correlation 

<div class="alert alert-success">Changed.</div>

In [ ]:
# Compute selected stats
dfinfo = pd.DataFrame(df.dtypes,columns=["dtypes"])
for (m,n) in zip([df.count(),df.isna().sum()],["count","isna"]):
    dfinfo = dfinfo.merge(pd.DataFrame(m,columns=[n]),right_index=True,left_index=True,how="inner");

#dfinfo.T.append(df.describe())
dfinfo = pd.concat([dfinfo.T, df.describe()])
dfinfo

In [ ]:
plt.figure(figsize=(9,4))
for (i,v) in enumerate(df.columns):
    plt.subplot(2,5,i+1);
    plt.hist(df.iloc[:,i],50,density=True)
    plt.legend([df.columns[i]],fontsize=6);

In [ ]:
df.corr().round(2).style.background_gradient(cmap="viridis")

Prepare for fitting by scaling data set
* Here, the data is scaled prior train-test split. 
    * In real applications, first split and scale afterwards, to simulate real-world scenario where we do not have the test set!

In [ ]:
X = StandardScaler().fit_transform(df.drop("target",axis=1).values);
y = df.target.values

#### Supervised Reduction
* Considering the correlation, histogram and the summary table:
    * Remove/drop "AveOccup" (average house occupancy)

<div class="alert alert-success">Changed.</div>

In [ ]:
#df = df.drop(["AveOccup"],axis=1)

In [ ]:
X = df.drop(columns=["target"])  
y = df["target"]

selector = SelectKBest(score_func=f_regression, k=5)  # Select top 5 features
X_selected = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features)

<div class="alert alert-success">Changed.</div>

In [ ]:
#PCA considering selected features

selected_feature_names = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Latitude']
X_selected = df[selected_feature_names]

pca = PCA(n_components="mle")  
X = pca.fit_transform(X_selected)

#Print the number of features after PCA
print(f"Number of features after PCA: {X.shape[1]} (input = {X_selected.shape[1]})")


#### Fit model

In [ ]:
[X_train,X_test,y_train,y_test] = train_test_split(X,y,train_size=0.67,test_size=0.33,random_state=123);

In [ ]:
knn = KNeighborsRegressor();

parameters = {"n_neighbors":[1,3,5,7,9],"weights":["uniform","distance"]}

knn_reg = GridSearchCV(knn, parameters, cv=5, scoring="neg_mean_squared_error");

In [ ]:
knn_reg.fit(X_train,y_train)

In [ ]:
knn_reg.best_estimator_

<div class="alert alert-success">Changed.</div>

In [ ]:
# from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

rmse = np.sqrt(mean_squared_error(knn_reg.predict(X_test),y_test))
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2_score(knn_reg.predict(X_test),y_test):.4f}")
print(f"Mean Absolute Error: {mean_absolute_error(knn_reg.predict(X_test),y_test):.4f}")
print(f"Mean Squared Error: {mean_squared_error(knn_reg.predict(X_test),y_test):.4f}")

print("Regression kNN (test) RMSE \t= {:.0f} *1000$".format(
    100*np.sqrt(mean_squared_error(knn_reg.predict(X_test),y_test))))

<div class="alert alert-success">Changed.</div>

In [ ]:
knn = KNeighborsRegressor()

param_grid = {
    "n_neighbors": list(range(1, 5, 1)),
    "weights": ["uniform", "distance"],
    "p": [1, 2] 
}

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="r2")  
grid_search.fit(X_train, y_train)

# Get best parameters
print("Best K:", grid_search.best_params_["n_neighbors"])
print("Best Weights:", grid_search.best_params_["weights"])

<div class="alert alert-success">Changed.</div>

In [ ]:
#import matplotlib.pyplot as plt
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.metrics import mean_squared_error

rmse_list = []
k_values = list(range(1, 10, 2)) 


for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k, weights="distance")
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    # Compute RMSE 
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))  
    rmse_list.append(rmse)

plt.figure(figsize=(8, 5))
plt.plot(k_values, rmse_list, marker="o", linestyle="-", color="blue")
plt.xlabel("k (Neighbors)")
plt.ylabel("RMSE")
plt.title("Elbow Method for Optimal k")
plt.show()






<div class="alert alert-success">Changed.</div>

## Application of kNN - Wine dataset

# Importing libraries
import numpy as np
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score

In [ ]:
# Load the Wine dataset

data = load_wine()
df = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

In [ ]:
# Split, train and predict k-NN model

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42, stratify=y)

# Train a basic k-NN 
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Predictions
y_pred = knn.predict(X_test)

In [ ]:
# Performance Evaluation
accuracy = accuracy_score(y_test, y_pred)
print(f"Initial Model Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Hyperparameter tuning using GridSearchCV
param_grid = {'n_neighbors': range(1, 31), 'weights': ['uniform', 'distance']}
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Best parameters
best_k = grid_search.best_params_['n_neighbors']
best_weights = grid_search.best_params_['weights']
print(f"Best k: {best_k}, Best weights: {best_weights}")

In [ ]:
# Train the optimized model
knn_optimized = KNeighborsClassifier(n_neighbors=best_k, weights=best_weights)
knn_optimized.fit(X_train, y_train)

# Predictions with optimized model
y_pred_optimized = knn_optimized.predict(X_test)

In [ ]:
# Performance Evaluation of optimized model
accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
print(f"Optimized Model Accuracy: {accuracy_optimized:.4f}")
print("Classification Report (Optimized Model):\n", classification_report(y_test, y_pred_optimized))
print("Confusion Matrix (Optimized Model):\n", confusion_matrix(y_test, y_pred_optimized))

In [ ]:
# Cross-validation score
cv_scores = cross_val_score(knn_optimized, X_train, y_train, cv=5)
print(f"Cross-validation mean accuracy: {cv_scores.mean():.4f}")

# Compare performance
print(f"Accuracy Improvement: {accuracy_optimized - accuracy:.4f}")

## Example - Regression KNN

In [ ]:
# Load the Diabetes dataset
data = load_diabetes()
X = data.data  # Features
y = data.target  # Target variable

# Split dataset (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Standardize features (Default)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Default KNN Model (k=5, Euclidean)
knn_default = KNeighborsRegressor(n_neighbors=5)
knn_default.fit(X_train_scaled, y_train)
y_pred_default = knn_default.predict(X_test_scaled)

In [ ]:
# Evaluate the model
mae = mean_absolute_error(y_test, y_pred_default)
mse = mean_squared_error(y_test, y_pred_default)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_default)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# KNN with Manhattan Distance
knn_manhattan = KNeighborsRegressor(n_neighbors=5, metric='manhattan')
knn_manhattan.fit(X_train_scaled, y_train)
y_pred_manhattan = knn_manhattan.predict(X_test_scaled)

In [ ]:
mae = mean_absolute_error(y_test, y_pred_manhattan)
mse = mean_squared_error(y_test, y_pred_manhattan)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_manhattan)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
# MinMax Scaling
scaler_minmax = MinMaxScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train)
X_test_minmax = scaler_minmax.transform(X_test)
knn_minmax = KNeighborsRegressor(n_neighbors=5)
knn_minmax.fit(X_train_minmax, y_train)
y_pred_minmax = knn_minmax.predict(X_test_minmax)

In [ ]:
mae = mean_absolute_error(y_test, y_pred_minmax)
mse = mean_squared_error(y_test, y_pred_minmax)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_minmax)

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

In [ ]:
models = ["Default KNN", "Manhattan", "MinMax Scaling"]
predictions = [y_pred_default, y_pred_manhattan, y_pred_minmax]

results = []
for model, y_pred in zip(models, predictions):
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    results.append([model, mae, mse, rmse, r2])

results_df = pd.DataFrame(results, columns=["Model", "MAE", "MSE", "RMSE", "R² Score"])

print("Model Performance Comparison:")
print(results_df.sort_values(by="R² Score", ascending=False))
